## Import Library

In [86]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


## Load Dataset

In [87]:
sleep_df = pd.read_csv(
    "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv",
    parse_dates=["SleepDay"],
    date_format="%m/%d/%Y %I:%M:%S %p"
)


steps_df = pd.read_csv(
    "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/hourlySteps_merged.csv",
    parse_dates=["ActivityHour"],
     date_format="%m/%d/%Y %I:%M:%S %p"
)

cal_df = pd.read_csv(
    "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/hourlyCalories_merged.csv",
    parse_dates=["ActivityHour"],
    date_format="%m/%d/%Y %I:%M:%S %p"
)

hr_df = pd.read_csv(
    "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv",
    parse_dates=["Time"],
    date_format="%m/%d/%Y %I:%M:%S %p"
)


In [88]:
sleep_df = sleep_df.sort_values(["Id", "SleepDay"])
steps_df = steps_df.sort_values(["Id", "ActivityHour"])
cal_df = cal_df.sort_values(["Id", "ActivityHour"])
hr_df = hr_df.sort_values(["Id", "Time"])

# Ensure datetime (extra safety)
hr_df["Time"] = pd.to_datetime(hr_df["Time"])

# ================================
# 3. RESAMPLE HEART RATE → HOURLY
# ================================

hr_hourly = (
    hr_df
    .set_index("Time")
    .groupby("Id")["Value"]
    .resample("h")        # ✅ lowercase h (future-proof)
    .mean()
    .reset_index()
    .rename(columns={"Value": "heart_rate"})
)

In [89]:
df = steps_df.merge(cal_df, on=["Id", "ActivityHour"])
df = df.merge(hr_hourly, left_on=["Id", "ActivityHour"], right_on=["Id", "Time"])
df.drop(columns=["Time"], inplace=True)

In [90]:
sleep_df["SleepDate"] = sleep_df["SleepDay"].dt.date
df["SleepDate"] = df["ActivityHour"].dt.date

df = df.merge(
    sleep_df[["Id", "SleepDate", "TotalMinutesAsleep"]],
    on=["Id", "SleepDate"],
    how="inner"
)

In [91]:
rhr = (
    df.groupby("Id")["heart_rate"]
    .quantile(0.25)
    .reset_index()
    .rename(columns={"heart_rate": "resting_hr"})
)

df = df.merge(rhr, on="Id")
df["stress"] = df["heart_rate"] - df["resting_hr"]
df = df.dropna()


In [92]:
df["sleep_score"] = np.clip((df["TotalMinutesAsleep"] / 480) * 100, 0, 100)
df["rhr_score"] = np.clip(100 - (df["resting_hr"] - 60) * 2, 0, 100)
df["hrv_score"] = np.clip(100 - abs(df["heart_rate"] - df["resting_hr"]), 0, 100)
df["recovery_score"] = np.clip(100 - df["stress"] * 5, 0, 100)

df["readiness_score"] = (
    0.30 * df["sleep_score"] +
    0.25 * df["recovery_score"] +
    0.20 * df["hrv_score"] +
    0.15 * df["rhr_score"]
)

In [93]:
FEATURES = ["StepTotal", "Calories", "heart_rate", "stress"]
TARGETS = ["sleep_score", "hrv_score", "rhr_score", "recovery_score", "readiness_score"]

scaler = MinMaxScaler()
df[FEATURES] = scaler.fit_transform(df[FEATURES])

In [94]:
SEQ_LEN = 6
X, y = [], []

for user in df["Id"].unique():
    user_df = df[df["Id"] == user].sort_values("ActivityHour")

    for i in range(len(user_df) - SEQ_LEN):
        X.append(user_df[FEATURES].iloc[i:i+SEQ_LEN].values)
        y.append(user_df[TARGETS].iloc[i+SEQ_LEN].values)

X = np.array(X)
y = np.array(y)

In [95]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# =========================
# MODEL
# =========================

input_layer = Input(shape=(SEQ_LEN, len(FEATURES)))

shared = LSTM(64, return_sequences=True, name="encoder_lstm_1")(input_layer)
shared = LSTM(32, name="encoder_lstm_2")(shared)
shared = Dropout(0.3)(shared)

sleep_out = Dense(1, name="sleep")(shared)
hrv_out = Dense(1, name="hrv")(shared)
rhr_out = Dense(1, name="rhr")(shared)
recovery_out = Dense(1, name="recovery")(shared)
readiness_out = Dense(1, name="readiness")(shared)

fitbit_model = Model(
    input_layer,
    [sleep_out, hrv_out, rhr_out, recovery_out, readiness_out]
)

# =========================
# COMPILE (BENAR)
# =========================

fitbit_model.compile(
    optimizer=Adam(0.001),
    loss={
        "sleep": "mse",
        "hrv": "mse",
        "rhr": "mse",
        "recovery": "mse",
        "readiness": "mse"
    },
    metrics={
        "sleep": "mae",
        "hrv": "mae",
        "rhr": "mae",
        "recovery": "mae",
        "readiness": "mae"
    }
)

# =========================
# FIT
# =========================

fitbit_model.fit(
    X,
    {
        "sleep": y[:, 0],
        "hrv": y[:, 1],
        "rhr": y[:, 2],
        "recovery": y[:, 3],
        "readiness": y[:, 4],
    },
    epochs=20,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - hrv_loss: 7839.0410 - hrv_mae: 87.8925 - loss: 33488.1289 - readiness_loss: 5158.2046 - readiness_mae: 70.5637 - recovery_loss: 4599.5542 - recovery_mae: 56.8026 - rhr_loss: 8423.0957 - rhr_mae: 91.3129 - sleep_loss: 7468.2285 - sleep_mae: 84.4968 - val_hrv_loss: 7120.9619 - val_hrv_mae: 84.1197 - val_loss: 29441.7266 - val_readiness_loss: 4407.6519 - val_readiness_mae: 65.6329 - val_recovery_loss: 3844.1470 - val_recovery_mae: 53.2731 - val_rhr_loss: 7112.4443 - val_rhr_mae: 84.0574 - val_sleep_loss: 6739.0610 - val_sleep_mae: 80.3178
Epoch 2/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - hrv_loss: 6915.2310 - hrv_mae: 82.4043 - loss: 29062.6973 - readiness_loss: 4343.0776 - readiness_mae: 64.4936 - recovery_loss: 3917.8784 - recovery_mae: 52.5669 - rhr_loss: 7423.3896 - rhr_mae: 85.6636 - sleep_loss: 6463.1226 - sleep_mae: 78.3181 - val_hrv_loss: 6538.8735 - val_hrv_mae: 80.5740 - val_loss: 26928.5566 - val_readiness_loss: 3960.9189 -

In [96]:
encoder = Model(
    inputs=fitbit_model.input,
    outputs=fitbit_model.get_layer("encoder_lstm_2").output
)

encoder.save("fitbit_encoder.keras")

## Fine Tuning

In [97]:
df = pd.read_csv(
    "/kaggle/input/smartwatch-clean-dataset/smartwatch_clean.csv"
)

In [98]:
df.head()

,heart_rate,blood_oxygen_level,step_count,sleep_duration,activity_level,stress_level
0,58.939776,98.809650,5450.390578,7.167236,Highly Active,1
1,74.883189,98.532195,727.601610,6.538239,Highly Active,5
2,247.803052,97.052954,2826.521994,6.515466,Highly Active,5
3,40.000000,96.894213,13797.338044,7.367790,Active,3
4,61.950165,98.583797,15679.067648,6.515466,Highly Active,6


In [99]:
from sklearn.preprocessing import LabelEncoder

dataolahencoding = df.copy() # Membuat salinan dataset untuk mencegah modifikasi dataset asli

# Label encoding 
encoder = LabelEncoder()
dataolahencoding['heart_rate_encod'] = encoder.fit_transform(dataolahencoding['heart_rate'])
dataolahencoding['blood_oxygen_level_encod'] = encoder.fit_transform(dataolahencoding['blood_oxygen_level'])
dataolahencoding['step_count_encod'] = encoder.fit_transform(dataolahencoding['step_count'])
dataolahencoding['sleep_duration_encod'] = encoder.fit_transform(dataolahencoding['sleep_duration'])
dataolahencoding['activity_level_encod'] = encoder.fit_transform(dataolahencoding['activity_level'])
dataolahencoding['stress_level_encod'] = encoder.fit_transform(dataolahencoding['stress_level'])

In [100]:
dataolahencoding = dataolahencoding.drop(['heart_rate', 'blood_oxygen_level', 'step_count', 'sleep_duration', 'activity_level','stress_level'], axis=1)
dataolahencoding.head(100)

,heart_rate_encod,blood_oxygen_level_encod,step_count_encod,sleep_duration_encod,activity_level_encod,stress_level_encod
0,1240,6185,5207,6330,1,0
1,4508,5703,947,4802,1,5
2,9299,3043,3220,4743,1,5
3,0,2772,8356,6747,0,3
4,1711,5789,8692,4743,1,6
...,...,...,...,...,...,...
95,0,707,7660,8342,0,4
96,7987,4604,116,5353,0,6
97,96,8012,3120,5408,0,6
98,9154,7804,4945,4903,2,1


In [101]:
dataolahencoding["sleep_score"] = np.clip((dataolahencoding["sleep_duration_encod"] / 8) * 100, 0, 100)
dataolahencoding["rhr_score"] = np.clip(100 - (dataolahencoding["heart_rate_encod"] - 60) * 2, 0, 100)
dataolahencoding["hrv_score"] = np.clip(100 - abs(dataolahencoding["heart_rate_encod"] - dataolahencoding["heart_rate_encod"].mean()), 0, 100)
dataolahencoding["recovery_score"] = np.clip(100 - dataolahencoding["stress_level_encod"] * 10, 0, 100)

dataolahencoding["readiness_score"] = (
    0.30 * dataolahencoding["sleep_score"] +
    0.25 * dataolahencoding["recovery_score"] +
    0.20 * dataolahencoding["hrv_score"] +
    0.15 * dataolahencoding["rhr_score"]
)


In [109]:
FEATURES = ["heart_rate_encod", "blood_oxygen_level_encod", "step_count_encod", "stress_level_encod"]
TARGETS = ["sleep_score", "hrv_score", "rhr_score", "recovery_score", "readiness_score"]

SEQ_LEN = 6
X, y = [], []

for i in range(len(dataolahencoding) - SEQ_LEN):
    X.append(dataolahencoding[FEATURES].iloc[i:i+SEQ_LEN].values)
    y.append(dataolahencoding[TARGETS].iloc[i+SEQ_LEN].values)

X = np.array(X)
y = np.array(y)


In [110]:
from tensorflow.keras.models import load_model

encoder = load_model("fitbit_encoder.keras")

for layer in encoder.layers:
    layer.trainable = False   # freeze encoder


In [112]:
input_layer = Input(shape=(SEQ_LEN, len(FEATURES)))
shared = encoder(input_layer)

shared = Dense(32, activation="relu")(shared)

outputs = [
    Dense(1, name="sleep")(shared),
    Dense(1, name="hrv")(shared),
    Dense(1, name="rhr")(shared),
    Dense(1, name="recovery")(shared),
    Dense(1, name="readiness")(shared)
]

final_model = Model(input_layer, outputs)

final_model.compile(
    optimizer=Adam(0.0005),
    loss="mse",
    metrics={
        "sleep": "mae",
        "hrv": "mae",
        "rhr": "mae",
        "recovery": "mae",
        "readiness": "mae"
    }
)


final_model.fit(
    X,
    [y[:,0], y[:,1], y[:,2], y[:,3], y[:,4]],
    epochs=20,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/20
245/245 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - hrv_loss: 199.6698 - hrv_mae: 3.5027 - loss: 15547.6631 - readiness_loss: 1728.6748 - readiness_mae: 40.6959 - recovery_loss: 3669.6023 - recovery_mae: 53.1427 - rhr_loss: 150.2372 - rhr_mae: 2.1563 - sleep_loss: 9799.4639 - sleep_mae: 98.9537 - val_hrv_loss: 161.0831 - val_hrv_mae: 4.3760 - val_loss: 11642.8975 - val_readiness_loss: 814.8217 - val_readiness_mae: 27.4255 - val_recovery_loss: 2413.1995 - val_recovery_mae: 40.8170 - val_rhr_loss: 190.5660 - val_rhr_mae: 3.5883 - val_sleep_loss: 8058.0444 - val_sleep_mae: 89.7666
Epoch 2/20
245/245 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - hrv_loss: 187.0610 - hrv_mae: 5.3566 - loss: 10091.1904 - readiness_loss: 553.2809 - readiness_mae: 21.4434 - recovery_loss: 2078.1401 - recovery_mae: 37.5977 - rhr_loss: 142.7758 - rhr_mae: 3.2440 - sleep_loss: 7129.9150 - sleep_mae: 84.2583 - val_hrv_loss: 161.1420 - val_hrv_mae: 5.4356 - val_loss: 5716.1377 - val_readiness_loss: 70.6556 - val_readiness_m